In [ ]:
#notebooks/01_Visor_folum.ipynb

# ------------------------------------------------------------------
# 0. IMPORTS Y CONFIG
# ------------------------------------------------------------------

import folium
from folium.plugins import MousePosition
import geopandas as gpd
from pathlib import Path
import sys
%load_ext autoreload
%autoreload 2
PROJECT_ROOT = Path().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT / 'scripts'))
from valencia_config import VALENCIA

PROJECT_ROOT = Path().resolve().parent
sys.path.insert(0, str(PROJECT_ROOT / 'scripts'))

# ------------------------------------------------------------------
# 1. LEER CUADRICULA DE TESELAS (EN EPSG:4326)
# ------------------------------------------------------------------

tiles_gdf = gpd.read_file(PROJECT_ROOT / 'data' / 'raw' / 'pnoa' / 'valencia_tiles_grid.geojson')

def create_map():
    m = folium.Map(
        location=VALENCIA['center_wgs'],
        tiles=None,
        width='100%', height='600px'
    )

    # Capa PNOA WMTS
    folium.TileLayer(
        tiles=(
            f"{VALENCIA['pnoa_url']}?"
            "service=WMTS&request=GetTile&version=1.0.0&"
            f"layer={VALENCIA['pnoa_layer']}&"
            "style=default&format=image/jpeg&"
            f"TileMatrixSet={VALENCIA['tilematrixset']}&"
            "TileMatrix={z}&TileRow={y}&TileCol={x}"
        ),
        attr='IGN PNOA 25cm',
        name='PNOA (base)',
        overlay=False,
        control=True,
        min_zoom=0, max_zoom=20
    ).add_to(m)

    # 2) Dibujar exactamente la geometría de cada tile
    for _, row in tiles_gdf.iterrows():
        folium.GeoJson(
            row.geometry.__geo_interface__,
            style_function=lambda _: {'color': 'red', 'weight': 2}
        ).add_to(m)

    MousePosition(position='bottomright').add_to(m)
    folium.LayerControl().add_to(m)
    
    # 3) Auto-ajuste del zoom para mostrar todo el grid
    minx, miny, maxx, maxy = tiles_gdf.total_bounds
    m.fit_bounds([[miny, minx], [maxy, maxx]])

    return m

# ------------------------------------------------------------------
# 2. GUARDAR Y MOSTRAR
# ------------------------------------------------------------------
# Guarda y muestra
mapa = create_map()
(PROJECT_ROOT / 'maps').mkdir(exist_ok=True)
mapa.save(str(PROJECT_ROOT / 'maps' / 'valencia_pnoa_tiles.html'))
mapa


## 🌍 Vista interactiva del Grid PNOA

👉 [Abrir el mapa interactivo de teselas de Valencia](./maps/valencia_pnoa_tiles.html)
